# Дембицкий М.Е. Домашнее задание № 2

## Оглавление
[Задание 1. Weighted Random Recommendation](#task1)  
[Задание 2. Расчет метрик](#task2)  
[Задание 3*. Улучшение бейзлайнов и ItemItem](#task3)  

In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [80]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [81]:
test_size_weeks = 3
data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

Создадим датафрейм с покупками юзеров на тестовом датасете (последние 3 недели)

In [82]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


### 1.1 Random recommendation

In [83]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [84]:
%%time

items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))
result.head(2)

Wall time: 4.9 s


,user_id,actual,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[914107, 6603404, 87493, 891234, 8160230]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[899970, 10356761, 2852520, 15511455, 1138917]"


### 1.2 Popularity-based recommendation

In [85]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [86]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)
result.head(2)

Wall time: 166 ms


,user_id,actual,random_recommendation,popular_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[914107, 6603404, 87493, 891234, 8160230]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[899970, 10356761, 2852520, 15511455, 1138917]","[6534178, 6533889, 1029743, 6534166, 1082185]"


<a name="task1"></a>
### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [87]:
# Cложить все значения sales_value, вес = значение sales_value / сумму. сформировать новый столбец.
popular = data.groupby('item_id')['sales_value'].sum().reset_index()
popular.sort_values('sales_value', ascending=False, inplace=True)

sales_value_sum = popular['sales_value'].sum()
popular['weight'] = popular['sales_value'] / sales_value_sum

items_weights = popular.drop('sales_value', axis=1)
items_weights.head()

,item_id,weight
56233,6534178,0.062970
56193,6533889,0.005738
29195,1029743,0.005111
56228,6534166,0.004211
35054,1082185,0.003672


In [88]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    
   
    
    # Сэмплируем товары по весам с помощью np.random.choices
    
    recs = np.random.choice(items_weights['item_id'], size=n, p=items_weights['weight'], replace=False)
    
    
    
    return recs.tolist()

In [89]:
%%time

# your_code

# Определяем самые популярные товары по sales_value
popular = data.groupby('item_id')['sales_value'].sum().reset_index()
popular.sort_values('sales_value', ascending=False, inplace=True)

# суммируем все значения sales_value,определяем вес каждого товара
sales_value_sum = popular['sales_value'].sum()
popular['weight'] = popular['sales_value'] / sales_value_sum

items_weights = popular.drop('sales_value', axis=1) 
    

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))
result.head(2)

Wall time: 2.45 s


,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[914107, 6603404, 87493, 891234, 8160230]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1110331, 7167968, 10149640, 6534178, 6396244]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[899970, 10356761, 2852520, 15511455, 1138917]","[6534178, 6533889, 1029743, 6534166, 1082185]","[9487987, 1106523, 10182848, 820798, 916122]"


<a name="task2"></a>
### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [90]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

# С файлом predictions_basic.csv что-то не так, он не работает. Я заметил что в колонке actual, item id указан без разделителя(запятых) - это я смог исправить, но он как не не работал, так и не работает.

In [91]:
bit_results = pd.read_csv('predictions_basic.csv')
bit_results.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [92]:
bit_result.apply(lambda row: precision_at_k(row['random_recommendation'], row['actual']), axis=1).mean()

IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed

In [93]:
# Попытка починить bit_results
bit_results['actual'] = result['actual']
bit_results.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [94]:
bit_result.apply(lambda row: precision_at_k(row['random_recommendation'], row['actual']), axis=1).mean()

IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed

In [95]:
# Было принято решение выполнять задание с моим файлом result и 3 алгоритмами рекомендаций
models = [
    'random_recommendation',
    'popular_recommendation',
    'weighted_random_recommendation'
]

for model in models:   
    print(f'{model} : {result.apply(lambda row: precision_at_k(row[model], row["actual"]), axis=1).mean()}')

random_recommendation : 0.0008814887365328109
popular_recommendation : 0.15523996082272082
weighted_random_recommendation : 0.021939275220372225


<a name="task3"></a>
### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [ ]:
# your_code